# Resources
https://docs.google.com/document/d/11MFXRQfpPkBS37qSlBGKb5EGD0DH7hEtKu-YlV5sK0I/edit

https://towardsdatascience.com/how-to-train-test-split-kfold-vs-stratifiedkfold-281767b93869?gi=194ee6e79b9d

In [ ]:
import re

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd

# We are also going to do some basic viz
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline


In [ ]:
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [ ]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'myersbriggs',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

connection = pg.connect(**connection_args)  # What is that "**" there??
cur = connection.cursor()

## Read data from .csv
```python
df = pd.read_csv('mbti_1.csv')
```

## Create a table in PostgreSQL database

```python
sqlCreateTable = "create table raw_data (id bigint, type varchar(4), posts text);"

cur.execute(sqlCreateTable)
connection.commit()
```

In [ ]:
cur.close()

## Inserting the data into the database.
```python
def single_insert(conn, insert_req, values):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req, values)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

    # Inserting each row
for i in df.index:

    query = "INSERT into raw_data(id, type, posts) values('%s',%s,%s);"
    values = (i, df['type'][i], df['posts'][i])
    single_insert(connection, query, values)
```

In [ ]:
# Close the connection
connection.close()

In [ ]:
postgreSQL_select_Query = "SELECT * FROM raw_data"

cur.execute(postgreSQL_select_Query)
print("Selecting rows from mobile table using cursor.fetchall")
myers_briggs = cur.fetchall() 


In [23]:
mb_df = pd.DataFrame(myers_briggs)
mb_df = mb_df.drop(0, axis=1)

post_list = [re.split('\|\|\|+', post) for post in mb_df[2]]
post_df = pd.DataFrame(post_list)